In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

# Law Article 13

## cu_13_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：结合所处行业特点、发展阶段和自身经营模式、盈利水平、偿债能力、资金需求等因素，未进行现金分红或现金分红水平较低原因的说明 |
| contextual_info | nan |
| note | 只考虑上市后年报分红情况; "本年"理解为上一个最新的年报的数据; 最近三年理解为检查上市后的最近三年年报情况, 上市不足三年的取上市后所有年的年报情况 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2512 |
| completion_tokens | 11821 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_1(df):
    '''
    检查cu_13_1的合规性：
    主体：上市公司
    条件：合并和母公司未分配利润均为正值且报告期内盈利，且未分红或最近三年分红不足
    约束：权益分派方案中披露原因说明
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_1_subject'] = True  # 所有行都是上市公司
    df['cu_13_1_condition'] = False
    df['cu_13_1_constraint'] = False

    # 转换日期格式
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    
    # 筛选年报数据（12月31日）
    annual_mask = df['财务数据日期'].dt.month == 12
    annual_df = df[annual_mask].copy()
    
    if not annual_df.empty:
        # 按公司分组处理
        annual_grouped = annual_df.groupby('公司简称', group_keys=False)
        
        # 计算三年滚动窗口
        def calculate_window(group):
            group = group.sort_values('财务数据日期')
            group['cash_dividend_3y_sum'] = group['现金分红金额'].rolling(3, min_periods=1).sum()
            group['net_profit_3y_avg'] = group['净利润'].rolling(3, min_periods=1).mean()
            return group
            
        annual_df = annual_grouped.apply(calculate_window)
        
        # 合并计算结果回原dataframe
        df = df.merge(
            annual_df[['公司简称', '财务数据日期', 'cash_dividend_3y_sum', 'net_profit_3y_avg']],
            on=['公司简称', '财务数据日期'],
            how='left'
        )
        
        # 条件判断
        cond_a = (df['合并报表未分配利润'] > 0) & (df['母公司报表未分配利润'] > 0)
        cond_b = df['净利润'] > 0
        cond_c = (~df['是否实施现金分红']) | (df['cash_dividend_3y_sum'] < df['net_profit_3y_avg'] * 0.3)
        
        # 组合条件并标记年报行
        df.loc[annual_mask, 'cu_13_1_condition'] = (cond_a & cond_b & cond_c)[annual_mask]

    # 约束条件检查（仅当满足主体和条件时检查）
    mask = df['cu_13_1_subject'] & df['cu_13_1_condition'] & df['是否进行权益分派']
    df.loc[mask, 'cu_13_1_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(
            text, 
            "结合所处行业特点、发展阶段和自身经营模式、盈利水平、偿债能力、资金需求等因素，"
            "未进行现金分红或现金分红水平较低原因的说明"
        )
    )

    return df

In [ ]:
df = check_cu_13_1(df)
df

---

## cu_13_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：留存未分配利润的预计用途以及收益情况 |
| contextual_info | nan |
| note | 只考虑上市后年报分红情况; "本年"理解为上一个最新的年报的数据; 最近三年理解为检查上市后的最近三年年报情况, 上市不足三年的取上市后所有年的年报情况 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2489 |
| completion_tokens | 9919 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_13_2(df):
    df = df.copy()
    # 初始化标记列
    df['cu_13_2_subject'] = True  # 所有上市公司均为责任主体
    df['cu_13_2_condition'] = False
    df['cu_13_2_constraint'] = None

    # 转换日期格式并筛选年报数据
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    is_annual_report = (df['财务数据日期'].dt.month == 12) & (df['财务数据日期'].dt.day == 31)
    
    # 预处理年报数据集
    annual_data = df[is_annual_report].copy()
    annual_data['年份'] = annual_data['财务数据日期'].dt.year

    # 按公司构建年报时序数据
    annual_records = {}
    for company in annual_data['公司简称'].unique():
        company_df = annual_data[annual_data['公司简称'] == company].sort_values('年份')
        annual_records[company] = company_df

    # 条件检查主逻辑
    for idx, row in df[is_annual_report].iterrows():
        # 基础财务条件验证
        profit_condition = (row['合并报表未分配利润'] > 0) and (row['母公司报表未分配利润'] > 0)
        profitability = row['净利润'] > 0
        
        if not (profit_condition and profitability):
            continue

        # 现金分红条件验证
        company_data = annual_records.get(row['公司简称'], pd.DataFrame())
        current_year = row['财务数据日期'].year
        
        # 获取上市后最近三年年报
        valid_years = company_data[(company_data['年份'] >= current_year-2) & (company_data['年份'] <= current_year)]
        
        # 计算分红约束指标
        if not row['是否实施现金分红']:
            df.loc[idx, 'cu_13_2_condition'] = True
        else:
            total_dividend = valid_years['现金分红金额'].sum()
            avg_profit = valid_years['净利润'].mean() if not valid_years.empty else 0
            if total_dividend < avg_profit * 0.3:
                df.loc[idx, 'cu_13_2_condition'] = True

    # 约束条件检查（仅对符合主体和条件的行）
    condition_mask = df['cu_13_2_subject'] & df['cu_13_2_condition']
    df.loc[condition_mask, 'cu_13_2_constraint'] = df.loc[condition_mask, '权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "留存未分配利润的预计用途以及收益情况") if pd.notna(x) else False
    )

    return df

In [ ]:
df = check_cu_13_2(df)
df

---

## cu_13_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：公司在相应期间是否按照中国证券监督管理委员会相关规定为中小股东参与现金分红决策提供了便利 |
| contextual_info | nan |
| note | 只考虑上市后年报分红情况; "本年"理解为上一个最新的年报的数据; 最近三年理解为检查上市后的最近三年年报情况, 上市不足三年的取上市后所有年的年报情况 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2498 |
| completion_tokens | 10105 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_13_3(df):
    '''
    检查cu_13_3的合规性：
    主体：上市公司
    条件：合并和母公司资产负债表未分配利润均为正，报告期内盈利，且未分红或最近三年分红总额低于年均净利润30%
    约束：权益分派方案中披露是否为中小股东参与现金分红决策提供便利
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_3_subject'] = True  # 所有公司都是上市公司
    df['cu_13_3_condition'] = False
    df['cu_13_3_constraint'] = None
    
    # 处理条件部分
    # 筛选年报数据行（财务数据日期为12月）
    is_annual = df['财务数据日期'].dt.month == 12
    annual_df = df[is_annual].copy()
    
    if not annual_df.empty:
        # 按公司分组并排序
        annual_df.sort_values(['公司简称', '财务数据日期'], inplace=True)
        
        # 计算前两年的现金分红和净利润
        annual_df['prev1_cash'] = annual_df.groupby('公司简称')['现金分红金额'].shift(1)
        annual_df['prev2_cash'] = annual_df.groupby('公司简称')['现金分红金额'].shift(2)
        annual_df['prev1_net'] = annual_df.groupby('公司简称')['净利润'].shift(1)
        annual_df['prev2_net'] = annual_df.groupby('公司简称')['净利润'].shift(2)
        
        # 计算现金分红总和和净利润总和
        annual_df['cash_sum'] = annual_df['现金分红金额'].fillna(0)
        annual_df['cash_sum'] += annual_df['prev1_cash'].fillna(0)
        annual_df['cash_sum'] += annual_df['prev2_cash'].fillna(0)
        
        annual_df['net_sum'] = annual_df['净利润'].fillna(0)
        annual_df['net_sum'] += annual_df['prev1_net'].fillna(0)
        annual_df['net_sum'] += annual_df['prev2_net'].fillna(0)
        
        # 计算实际年数
        annual_df['year_count'] = 1
        annual_df['year_count'] += (~annual_df['prev1_cash'].isnull()).astype(int)
        annual_df['year_count'] += (~annual_df['prev2_cash'].isnull()).astype(int)
        
        # 计算年均净利润
        annual_df['avg_net'] = annual_df['net_sum'] / annual_df['year_count'].replace(0, 1)  # 防止除零
        
        # 现金分红条件：未分红或总和 < 年均净利润的30%
        condition_cash = (annual_df['cash_sum'] == 0) | (annual_df['cash_sum'] < (annual_df['avg_net'] * 0.3))
        
        # 其他条件：未分配利润均为正，净利润为正
        condition_met = (
            (annual_df['合并报表未分配利润'] > 0) &
            (annual_df['母公司报表未分配利润'] > 0) &
            (annual_df['净利润'] > 0) &
            condition_cash
        )
        
        # 将条件结果合并回原df
        annual_df['condition_met'] = condition_met
        merge_keys = ['公司简称', '财务数据日期']
        merged = df.merge(
            annual_df[merge_keys + ['condition_met']],
            on=merge_keys,
            how='left',
            suffixes=('', '_annual')
        )
        df['cu_13_3_condition'] = merged['condition_met'].fillna(False)
    
    # 处理约束部分
    # 仅当subject和condition为True，且进行了权益分派时检查
    mask = (df['cu_13_3_subject']) & (df['cu_13_3_condition']) & (df['是否进行权益分派'])
    df.loc[mask, 'cu_13_3_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, "公司在相应期间是否按照中国证券监督管理委员会相关规定为中小股东参与现金分红决策提供了便利")
    )
    
    return df

In [ ]:
df = check_cu_13_3(df)
df

---

## cu_13_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：公司为增强投资者回报水平拟采取的措施 |
| contextual_info | nan |
| note | 只考虑上市后年报分红情况; "本年"理解为上一个最新的年报的数据; 最近三年理解为检查上市后的最近三年年报情况, 上市不足三年的取上市后所有年的年报情况 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2488 |
| completion_tokens | 8395 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_13_4(df):
    '''
    检查cu_13_4的合规性：
    主体：上市公司
    条件：合并/母公司未分配利润为正+报告期盈利+分红不足
    约束：权益分派方案需披露增强回报措施
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_4_subject'] = True  # 所有行均为上市公司
    df['cu_13_4_condition'] = False
    df['cu_13_4_constraint'] = None

    # 预处理步骤：获取各公司最新年报数据
    annual_mask = df['财务数据日期'].dt.month == 12
    annual_reports = df[annual_mask].sort_values(['公司简称', '财务数据日期'])
    
    # 获取每个公司最新年报数据
    latest_annual = annual_reports.groupby('公司简称').tail(1)
    df = df.merge(
        latest_annual[['公司简称', '合并报表未分配利润', '母公司报表未分配利润', '净利润']]
            .rename(columns={
                '合并报表未分配利润': 'latest_合并未分配利润',
                '母公司报表未分配利润': 'latest_母公司未分配利润',
                '净利润': 'latest_净利润'
            }),
        on='公司简称',
        how='left'
    )

    # 预处理步骤：计算最近三年分红数据
    def get_three_year_data(group):
        return group.tail(3)  # 取最近三年
    
    three_year_reports = annual_reports.groupby('公司简称', group_keys=False).apply(get_three_year_data)
    
    # 计算分红总额和年均净利润
    dividend_sum = three_year_reports.groupby('公司简称')['现金分红金额'].sum().reset_index(name='三年分红总额')
    profit_avg = three_year_reports.groupby('公司简称')['净利润'].mean().reset_index(name='年均净利润')
    
    df = df.merge(dividend_sum, on='公司简称', how='left')
    df = df.merge(profit_avg, on='公司简称', how='left')

    # 填充缺失值
    df['三年分红总额'] = df['三年分红总额'].fillna(0)
    df['年均净利润'] = df['年均净利润'].fillna(float('inf'))  # 避免除零错误

    # 条件判断
    condition_1 = (df['latest_合并未分配利润'] > 0) & (df['latest_母公司未分配利润'] > 0)
    condition_2 = df['latest_净利润'] > 0
    condition_3 = (df['三年分红总额'] == 0) | (df['三年分红总额'] < df['年均净利润'] * 0.3)
    
    df['cu_13_4_condition'] = condition_1 & condition_2 & condition_3

    # 约束条件检查（仅当进行权益分派时）
    constraint_mask = df['cu_13_4_condition'] & df['是否进行权益分派']
    df.loc[constraint_mask, 'cu_13_4_constraint'] = df.loc[constraint_mask, '权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "为增强投资者回报水平拟采取的措施") if pd.notnull(x) else False
    )

    # 清理中间列
    df.drop(columns=[
        'latest_合并未分配利润', 'latest_母公司未分配利润', 'latest_净利润',
        '三年分红总额', '年均净利润'
    ], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_cu_13_4(df)
df

---

## cu_13_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 母公司资产负债表中未分配利润为负值但合并资产负债表中未分配利润为正值 |
| constraint | 应当在权益分派相关公告中披露上市公司控股子公司向母公司实施利润分配的情况 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2398 |
| completion_tokens | 4939 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_13_5(df):
    '''
    检查cu_13_5的合规性：
    主体：上市公司（默认全部符合）
    条件：母公司未分配利润为负且合并报表未分配利润为正
    约束：在权益分派公告中披露子公司向母公司分配利润的情况
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_5_subject'] = True  # 根据要求主体默认全部为True
    df['cu_13_5_condition'] = (df['母公司报表未分配利润'] < 0) & (df['合并报表未分配利润'] > 0)
    df['cu_13_5_constraint'] = None
    
    # 确定需要检查约束的行（同时满足主体和条件）
    mask = df['cu_13_5_subject'] & df['cu_13_5_condition']
    
    # 定义需要检查的披露内容
    constraint_description = "披露上市公司控股子公司向母公司实施利润分配的情况"
    
    # 对符合条件的行进行约束检查
    df.loc[mask, 'cu_13_5_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, constraint_description) if pd.notnull(text) else False
    )
    
    # 处理文本为空的情况（视为未披露）
    df.loc[mask & df['权益分派方案文本'].isna(), 'cu_13_5_constraint'] = False
    
    return df

In [ ]:
df = check_cu_13_5(df)
df

---

## cu_13_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 母公司资产负债表中未分配利润为负值但合并资产负债表中未分配利润为正值 |
| constraint | 应当在权益分派相关公告中披露公司为增强投资者回报水平拟采取的措施 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2398 |
| completion_tokens | 2933 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_13_6(df):
    '''
    检查cu_13_6的合规性：
    主体：上市公司（默认全部符合）
    条件：母公司未分配利润<0且合并报表未分配利润>0
    约束：权益分派公告需披露增强回报措施
    '''
    df = df.copy()
    
    # 1. 标记责任主体（所有行均为上市公司）
    df['cu_13_6_subject'] = True
    
    # 2. 标记触发条件
    df['cu_13_6_condition'] = (
        (df['母公司报表未分配利润'] < 0) & 
        (df['合并报表未分配利润'] > 0)
    )
    
    # 3. 初始化约束列
    df['cu_13_6_constraint'] = None
    
    # 筛选需要检查约束的情形：主体条件符合且实施权益分派
    mask = (
        df['cu_13_6_subject'] & 
        df['cu_13_6_condition'] & 
        df['是否进行权益分派']
    )
    
    # 对符合条件的记录进行内容检查
    constraint_desc = "为增强投资者回报水平拟采取的措施"
    df.loc[mask, 'cu_13_6_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, constraint_desc) if pd.notna(text) else False
    )
    
    return df

In [ ]:
df = check_cu_13_6(df)
df

---

# Law Article 14

## cu_14_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，且最近连续两个会计年度经审计的下列财务报表项目金额合计占总资产的50%以上：交易性金融资产、衍生金融资产（套期保值工具除外）、债权投资、其他债权投资、其他权益工具投资、其他非流动金融资产、其他流动资产（待抵扣增值税、预缴税费、合同取得成本等与经营活动相关的资产除外），且未进行现金分红或者现金分红总额低于当年净利润50% |
| constraint | 应当在权益分派方案中结合前述财务报表列报项目的具体情况说明现金分红方案确定的依据及未来增强投资者回报的规划 |
| contextual_info | 财务报表项目具体范围包含：交易性金融资产、衍生金融资产（套期保值工具除外）、债权投资、其他债权投资、其他权益工具投资、其他非流动金融资产、其他流动资产（待抵扣增值税、预缴税费、合同取得成本等与经营活动相关的资产除外） |
| note | 主体默认为True, 不区分在上市前还是上市后; 只考虑年报情况; 数据已经排除在contextual_info中提及的除外项目 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2622 |
| completion_tokens | 6019 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_14_1(df):
    df = df.copy()
    
    # 初始化标记列
    df['cu_14_1_subject'] = True  # 上市公司默认全部符合主体要求
    df['cu_14_1_condition'] = False
    df['cu_14_1_constraint'] = None
    
    # 条件1: 未分配利润正值校验
    profit_condition = (
        (df['合并报表未分配利润'] > 0) & 
        (df['母公司报表未分配利润'] > 0)
    )
    
    # 条件2: 报告期盈利校验
    profitability_condition = (df['净利润'] > 0)
    
    # 条件3: 金融资产占比校验（仅处理年报数据）
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    is_annual_report = df['财务数据日期'].dt.month == 12
    
    # 计算金融资产合计（已排除例外项目）
    financial_assets = df[is_annual_report].eval(
        '交易性金融资产 + 衍生金融资产 + 债权投资 + 其他债权投资 + '
        '其他权益工具投资 + 其他非流动金融资产 + 其他流动资产'
    )
    
    # 按公司分组计算连续两年占比
    def check_consecutive_years(group):
        group = group.sort_values('财务数据日期')
        group['prev_year_ratio'] = group['金融资产总额'].shift(1) / group['总资产'].shift(1)
        group['current_year_ratio'] = group['金融资产总额'] / group['总资产']
        return (group['prev_year_ratio'] >= 0.5) & (group['current_year_ratio'] >= 0.5)
    
    ratio_condition = df[is_annual_report].groupby('公司简称', group_keys=False).apply(check_consecutive_years)
    df.loc[ratio_condition.reindex(df.index, fill_value=False).index, 'ratio_flag'] = True
    
    # 条件4: 分红合规校验
    dividend_condition = (
        (~df['是否实施现金分红']) | 
        (df['现金分红金额'] < df['净利润'] * 0.5)
    )
    
    # 综合条件判断
    final_condition = (
        profit_condition & 
        profitability_condition & 
        df['ratio_flag'].fillna(False) & 
        dividend_condition
    )
    df.loc[final_condition, 'cu_14_1_condition'] = True
    
    # 约束条件检查（仅对符合主体和条件的记录）
    mask = df['cu_14_1_subject'] & df['cu_14_1_condition']
    if mask.any():
        constraint_desc = "结合交易性金融资产、衍生金融资产等财务报表列报项目的具体情况说明现金分红方案确定的依据及未来增强投资者回报的规划"
        df.loc[mask, 'cu_14_1_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
            lambda x: llm_content_check(x, constraint_desc) if pd.notnull(x) else False
        )
    
    # 清理中间列
    df.drop(columns=['ratio_flag'], errors='ignore', inplace=True)
    
    return df

In [ ]:
df = check_cu_14_1(df)
df

---

# Law Article 15

## cu_15_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 权益分派方案中现金分红的金额达到或者超过当期净利润的100%且达到或者超过当期末未分配利润的50% |
| constraint | 应当同时披露是否影响偿债能力、过去十二个月内是否使用过募集资金补充流动资金以及未来十二个月内是否计划使用募集资金补充流动资金等内容 |
| contextual_info | nan |
| note | 未分配利润以合并报表的为准; 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2480 |
| completion_tokens | 2664 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_15_1(df):
    df = df.copy()
    
    # 初始化标记列
    df['cu_15_1_subject'] = True  # 所有公司均为上市公司
    df['cu_15_1_condition'] = False
    df['cu_15_1_constraint'] = None
    
    # 条件验证：现金分红金额双指标达标
    cash_condition = (
        df['是否实施现金分红'] &  # 必须实际实施现金分红
        (df['现金分红金额'] >= df['净利润']) &  # 达到净利润100%
        (df['现金分红金额'] >= df['合并报表未分配利润'] * 0.5)  # 达到未分配利润50%
    )
    df.loc[cash_condition, 'cu_15_1_condition'] = True
    
    # 约束验证：仅对触发条件的记录检查披露内容
    mask = df['cu_15_1_condition']  # 主体默认为True无需重复判断
    constraint_desc = "是否影响偿债能力、过去十二个月内是否使用过募集资金补充流动资金以及未来十二个月内是否计划使用募集资金补充流动资金"
    
    df.loc[mask, 'cu_15_1_constraint'] = df.loc[mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, constraint_desc) if pd.notna(text) else False
    )
    
    return df

In [ ]:
df = check_cu_15_1(df)
df

---

## cu_15_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 最近一个会计年度的财务会计报告被出具非无保留意见的审计报告或者带与持续经营相关的重大不确定性段落的无保留意见的审计报告，且实施现金分红的 |
| constraint | 应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营 |
| contextual_info | nan |
| note | 对应到我们数据库中记录的类型: 保留意见, 非标准无保留意见. 主体默认为True, 不区分在上市前还是上市后 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2453 |
| completion_tokens | 3039 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_15_2(df):
    '''
    检查cu_15_2的合规性：
    主体：上市公司
    条件：最近会计年度审计意见为非无保留意见或带持续经营重大不确定性段落，且实施现金分红
    约束：披露现金分红方案的合理性及对营运资金和生产经营的影响
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_15_2_subject'] = True  # 所有公司均为上市公司
    df['cu_15_2_condition'] = False
    df['cu_15_2_constraint'] = None

    # 2. 验证触发条件（独立标记）
    # 审计意见类型检查（保留意见/非标无保留）
    audit_condition = df['审计意见'].isin(['保留意见', '非标准无保留意见'])
    # 现金分红实施检查
    dividend_condition = df['是否实施现金分红']
    df['cu_15_2_condition'] = audit_condition & dividend_condition

    # 3. 验证约束条件（仅当主体和条件都满足时）
    valid_mask = df['cu_15_2_subject'] & df['cu_15_2_condition']
    constraint_desc = "根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营"
    
    # 使用LLM进行文本内容核查
    df.loc[valid_mask, 'cu_15_2_constraint'] = df.loc[valid_mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, constraint_desc) if pd.notnull(text) else False
    )

    return df

In [ ]:
df = check_cu_15_2(df)
df

---

## cu_15_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 报告期末资产负债率超过80%且当期经营活动产生的现金流量净额为负，且现金分红金额超过当期净利润50%的 |
| constraint | 应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_31_2 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2469 |
| completion_tokens | 1758 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_15_3(df):
    '''
    检查cu_15_3的合规性：
    主体：上市公司（默认全部符合）
    条件：资产负债率>80% & 经营现金流为负 & 现金分红>净利润50%
    约束：需披露分红方案合理性分析
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_15_3_subject'] = True  # 根据要求主体默认为True
    df['cu_15_3_condition'] = False
    df['cu_15_3_constraint'] = None

    # 1. 验证触发条件（三要素同时满足）
    # 条件1：资产负债率超过80%
    cond_leverage = df['资产负债率'] > 0.8
    
    # 条件2：经营活动现金流为负
    cond_cashflow = df['经营活动现金流量净额'] < 0
    
    # 条件3：现金分红超过净利润50%（需确保净利润为正）
    cond_dividend = (df['现金分红金额'] > 0.5 * df['净利润']) & (df['净利润'] > 0)
    
    # 合并条件
    df['cu_15_3_condition'] = cond_leverage & cond_cashflow & cond_dividend

    # 2. 验证约束条件（仅对符合条件的主体检查）
    mask = df['cu_15_3_condition']
    constraint_description = "根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营"
    
    # 仅对符合条件的记录进行内容检查
    df.loc[mask, 'cu_15_3_constraint'] = df[mask].apply(
        lambda row: llm_content_check(row['权益分派方案文本'], constraint_description) 
        if pd.notnull(row['权益分派方案文本']) else False,
        axis=1
    )

    return df

In [ ]:
df = check_cu_15_3(df)
df

---

# Law Article 18

## cu_18_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且最近两年同期净利润持续增长 |
| constraint | 每股送转股比例不得高于上市公司最近两年同期净利润的复合增长率 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后; 按照refer的单元的要求, 应当只处理财务数据为年报或半年报的记录 |
| relation | refer_to |
| target | MEU_31_2;MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2467 |
| completion_tokens | 11964 |


### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_cu_18_1(df):
    '''
    检查cu_18_1的合规性：
    主体：上市公司（所有行标记为True）
    条件：披露高送转方案且最近两年同期净利润持续增长
    约束：送转比例不超过净利润复合增长率
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_18_1_subject'] = True
    df['cu_18_1_condition'] = False
    df['cu_18_1_constraint'] = False
    
    # 转换财务数据日期格式
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    
    # 1. 条件验证准备
    # 确定财务期间类型（年报/半年报）
    df['财务期间类型'] = df['财务数据日期'].dt.month.apply(
        lambda m: '年报' if m == 12 else '半年报' if m == 6 else '其他'
    )
    valid_period_mask = df['财务期间类型'].isin(['年报', '半年报'])
    
    # 2. 条件验证
    # 条件一：实施高送转
    cond_high_transfer = df['是否实施股票股利'] | df['是否公积金转增股本']
    
    # 按公司和期间类型分组排序
    grouped = df.groupby(['公司简称', '财务期间类型'], group_keys=False)
    df_sorted = grouped.apply(lambda g: g.sort_values('财务数据日期'))
    
    # 计算前两期净利润
    df_sorted['prev1_net'] = grouped['净利润'].shift(1)
    df_sorted['prev2_net'] = grouped['净利润'].shift(2)
    
    # 合并回原始数据
    df = df.merge(
        df_sorted[['公司简称', '财务数据日期', 'prev1_net', 'prev2_net']],
        on=['公司简称', '财务数据日期'],
        how='left'
    )
    
    # 条件二：净利润连续增长
    cond_growth = (df['净利润'] > df['prev1_net']) & (df['prev1_net'] > df['prev2_net'])
    cond_growth = cond_growth.fillna(False)
    
    # 综合条件标记
    df.loc[valid_period_mask & cond_high_transfer & cond_growth, 'cu_18_1_condition'] = True
    
    # 3. 约束验证
    # 计算送转比例
    df['送转比例'] = (df['送股比例'].fillna(0) + df['转增比例'].fillna(0)) / 10
    
    # 计算复合增长率
    with np.errstate(divide='ignore', invalid='ignore'):
        growth_rate = np.sqrt(df['净利润'] / df['prev2_net']) - 1
    
    # 有效性检查（正增长且分母不为零）
    valid_growth = (df['prev2_net'] > 0) & (df['净利润'] > 0) & df['prev2_net'].notna()
    
    # 约束条件判断
    constraint_met = (df['送转比例'] <= growth_rate) & valid_growth
    df['cu_18_1_constraint'] = constraint_met.fillna(False)
    
    return df

In [ ]:
df = check_cu_18_1(df)
df

---

## cu_18_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且在报告期内实施再融资、并购重组等导致净资产有较大变化 |
| constraint | 每股送转股比例不得高于上市公司报告期末净资产较之于期初净资产的增长率 |
| contextual_info | nan |
| note | 简化假设: 不考虑年报季报的区别, 所有的披露都是单季度的口径. 只考虑本期和上期. 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2476 |
| completion_tokens | 6612 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_18_2(df):
    '''
    检查cu_18_2的合规性：
    主体：上市公司（默认全部成立）
    条件：披露高送转且报告期净资产发生显著变化
    约束：送转比例不超过净资产增长率
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_18_2_subject'] = True  # 所有行主体自动成立
    df['cu_18_2_condition'] = False
    df['cu_18_2_constraint'] = False

    # 计算净资产增长率（按公司分组滚动计算）
    df['净资产_上期'] = df.groupby('公司简称')['净资产'].shift(1)
    df['净资产增长率'] = (df['净资产'] - df['净资产_上期']) / df['净资产_上期']

    # 1. 条件验证（需同时满足两个条件）
    # 条件1：存在高送转方案（股票股利或转增股本）
    has_high_distribution = df['是否实施股票股利'] | df['是否公积金转增股本']
    
    # 条件2：净资产发生显著变化（假设增长率绝对值≥5%为显著）
    significant_change = df['净资产增长率'].abs() >= 0.05
    
    df.loc[has_high_distribution & significant_change, 'cu_18_2_condition'] = True

    # 2. 约束验证（所有行独立检查）
    # 计算每股送转比例（每10股转送数转换为每股比例）
    df['每股送转比例'] = (df['送股比例'] + df['转增比例']) / 10.0
    
    # 约束条件：送转比例≤净资产增长率（处理NaN和负增长率情况）
    valid_constraint = (
        (df['每股送转比例'] <= df['净资产增长率']) & 
        df['净资产增长率'].notna() &
        (df['净资产增长率'] > 0)  # 仅当增长率为正时约束有意义
    )
    df['cu_18_2_constraint'] = valid_constraint.fillna(False)

    # 清理中间列
    df.drop(columns=['净资产_上期', '每股送转比例'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_18_2(df)
df

---

## cu_18_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且最近两年净利润持续增长且最近三年每股收益均不低于1元且上市公司认为确有必要且依据年度财务报表进行高送转 |
| constraint | 应当充分披露高送转的主要考虑及其合理性且送转股后每股收益不低于0.5元 |
| contextual_info | nan |
| note | "每股收益"简化为采用"每股净利润";主体默认为True, 不区分在上市前还是上市后; 只考虑年报的情况 |
| relation | refer_to |
| target | MEU_31_2;MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2491 |
| completion_tokens | 10356 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_cu_18_3(df):
    '''
    检查cu_18_3的合规性：
    主体：上市公司（默认全部符合）
    条件：需同时满足：
        1. 年报披露高送转方案（股票股利或公积金转增）
        2. 最近两年净利润持续增长（逐年递增）
        3. 最近三年每股收益≥1元
        4. 公告包含必要性说明
    约束：需同时满足：
        1. 披露合理性说明
        2. 送转后每股收益≥0.5元
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_18_3_subject'] = True
    df['cu_18_3_condition'] = False
    df['cu_18_3_constraint'] = None

    # 预处理财务日期
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    is_annual = df['财务数据日期'].dt.month == 12
    has_high_sz = df['是否实施股票股利'] | df['是否公积金转增股本']

    # 按公司分组计算历史财务指标
    original_index = df.index
    df_sorted = df.sort_values(['公司简称', '财务数据日期'])
    
    # 计算历史净利润和每股收益
    df_sorted['prev1_net'] = df_sorted.groupby('公司简称')['净利润'].shift(1)
    df_sorted['prev2_net'] = df_sorted.groupby('公司简称')['净利润'].shift(2)
    df_sorted['prev1_eps'] = df_sorted.groupby('公司简称')['每股净利润'].shift(1)
    df_sorted['prev2_eps'] = df_sorted.groupby('公司简称')['每股净利润'].shift(2)
    
    # 标记净利润持续增长
    net_growth = (df_sorted['净利润'] > df_sorted['prev1_net']) & (df_sorted['prev1_net'] > df_sorted['prev2_net'])
    
    # 标记三年EPS达标
    eps_valid = (df_sorted['每股净利润'] >= 1) & (df_sorted['prev1_eps'] >= 1) & (df_sorted['prev2_eps'] >= 1)
    
    # 回填计算结果
    df_sorted = df_sorted.set_index(original_index)
    df['net_growth'] = df_sorted['net_growth'].reindex(df.index, fill_value=False)
    df['eps_valid'] = df_sorted['eps_valid'].reindex(df.index, fill_value=False)

    # 必要性检查（仅高送转年报）
    df['necessary_check'] = False
    mask = is_annual & has_high_sz
    df.loc[mask, 'necessary_check'] = df[mask]['权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "上市公司认为确有必要实施高送转的说明")
    )

    # 组合条件
    df['cu_18_3_condition'] = (
        is_annual &
        has_high_sz &
        df['net_growth'] &
        df['eps_valid'] &
        df['necessary_check']
    )

    # 约束检查（仅对符合条件的主体）
    mask = df['cu_18_3_condition']
    
    # 合理性披露检查
    df['disclosure_check'] = False
    df.loc[mask, 'disclosure_check'] = df[mask]['权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "充分披露高送转的主要考虑及其合理性")
    )
    
    # 计算送转后EPS
    sz_ratio = (df['送股比例'] + df['转增比例']) / 10
    df['post_sz_eps'] = df['每股净利润'] / (1 + sz_ratio)
    eps_constraint = df['post_sz_eps'] >= 0.5
    
    # 组合约束条件
    df.loc[mask, 'cu_18_3_constraint'] = df[mask]['disclosure_check'] & eps_constraint[mask]

    return df

In [ ]:
df = check_cu_18_3(df)
df

---

# Law Article 19

## cu_19_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 报告期尚未产生收入 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | "收入"见"营业收入";主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2427 |
| completion_tokens | 3592 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_1(df):
    '''
    检查cu_19_1的合规性：
    主体：上市公司（所有行均为True）
    条件：报告期营业收入为0
    约束：未实施高送转（送转股比例合计<5股）
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_19_1_subject'] = True  # 所有公司均为上市公司
    df['cu_19_1_condition'] = (df['营业收入'] == 0)
    
    # 计算高送转指标（每10股送转合计≥5股）
    has_equity_plan = df['是否实施股票股利'] | df['是否公积金转增股本']
    total_ratio = df['送股比例'] + df['转增比例']
    is_high_ratio = total_ratio >= 5
    
    # 标记约束条件（存在高送转方案则违规）
    df['cu_19_1_constraint'] = ~(has_equity_plan & is_high_ratio)

    return df

In [ ]:
df = check_cu_19_1(df)
df

---

## cu_19_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 净利润为负 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_31_2;MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2429 |
| completion_tokens | 8227 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_2(df):
    '''
    检查cu_19_2的合规性：
    主体：上市公司（所有行均为True）
    条件：净利润为负
    约束：不得披露高送转方案（送股比例+转增比例>=5视为高送转）
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_19_2_subject'] = True  # 所有公司均为上市公司
    df['cu_19_2_condition'] = df['净利润'] < 0
    df['cu_19_2_constraint'] = True  # 默认合规

    # 计算送转比例总和（每10股送转合计）
    total_ratio = df['送股比例'] + df['转增比例']
    
    # 识别高送转方案（权益分派且送转比例≥5）
    high_transfer = df['是否进行权益分派'] & (total_ratio >= 5)
    
    # 标记违规情形
    df.loc[high_transfer, 'cu_19_2_constraint'] = False

    return df

In [ ]:
df = check_cu_19_2(df)
df

---

## cu_19_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 净利润同比下降50%以上 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | 主体默认为True, 不区分在上市前还是上市后; 净利润下降的检查考虑单季度同比 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2431 |
| completion_tokens | 9561 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_3(df):
    '''
    检查cu_19_3的合规性：
    主体：上市公司（默认全部符合）
    条件：单季度净利润同比下降50%以上
    约束：不得披露高送转方案（送转比例合计每10股≥5股）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_19_3_subject'] = True  # 所有行主体均为上市公司
    df['cu_19_3_condition'] = False
    df['cu_19_3_constraint'] = True  # 默认未披露高送转
    
    # 1. 处理条件：净利润同比下降50%以上（单季度同比）
    # 按公司和财务数据日期排序计算同比
    original_index = df.index
    df_sorted = df.sort_values(['公司简称', '财务数据日期'])
    
    # 计算去年同季净利润（向前滚动4个季度）
    df_sorted['去年同季净利润'] = df_sorted.groupby('公司简称')['净利润'].shift(4)
    
    # 计算同比降幅（仅处理去年净利润为正的情况）
    mask = (
        (df_sorted['去年同季净利润'] > 0) &
        ((df_sorted['去年同季净利润'] - df_sorted['净利润']) / df_sorted['去年同季净利润'] >= 0.5)
    )
    df_sorted.loc[mask, 'cu_19_3_condition'] = True
    
    # 恢复原始索引顺序
    df = df_sorted.sort_index()
    
    # 2. 处理约束：高送转方案判断
    # 计算总送转比例（送股+转增）并检查是否实施
    total_ratio = df['送股比例'] + df['转增比例']
    implemented = df['是否实施股票股利'] | df['是否公积金转增股本']
    df.loc[implemented & (total_ratio >= 5), 'cu_19_3_constraint'] = False
    
    return df

In [ ]:
df = check_cu_19_3(df)
df

---

## cu_19_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 送转股后每股收益低于0.2元 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | "每股收益"简化为采用"每股净利润"; 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2438 |
| completion_tokens | 6055 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_4(df):
    '''
    检查cu_19_4的合规性：
    主体：上市公司（所有行均为True）
    条件：送转股后每股收益低于0.2元
    约束：不得披露高送转方案
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_19_4_subject'] = True  # 所有公司均为上市公司
    df['cu_19_4_condition'] = False
    df['cu_19_4_constraint'] = True  # 默认约束成立

    # 1. 条件判断：送转股后每股收益<0.2元
    # 仅当存在送转股行为时进行计算
    has_stock_dividend = df['是否实施股票股利'] | df['是否公积金转增股本']
    
    # 计算股本扩张比例（每10股送转合计股数/10）
    expansion_ratio = (df['送股比例'] + df['转增比例']) / 10
    
    # 计算送转后每股收益
    eps_after = df['每股净利润'] / (1 + expansion_ratio)
    
    # 标记条件成立的情况
    condition_mask = has_stock_dividend & (eps_after < 0.2)
    df.loc[condition_mask, 'cu_19_4_condition'] = True

    # 2. 约束判断：条件成立时不得实施高送转
    # 当且仅当条件成立且实际存在送转行为时违反约束
    violation_mask = df['cu_19_4_condition'] & has_stock_dividend
    df.loc[violation_mask, 'cu_19_4_constraint'] = False

    return df

In [ ]:
df = check_cu_19_4(df)
df

---

## cu_19_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 相关股东在前三个月存在减持情形 |
| constraint | 不得披露高送转方案 |
| contextual_info | 相关股东指公司的提议股东和控股股东及其一致行动人、董事、监事及高级管理人员 |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2443 |
| completion_tokens | 1968 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_5(df):
    '''
    检查cu_19_5的合规性：
    主体：上市公司（默认全部满足）
    条件：相关股东前三个月存在减持情形
    约束：不得披露高送转方案（股票股利/转增股本）
    '''
    df = df.copy()

    # 1. 标记责任主体（所有公司均为上市公司）
    df['cu_19_5_subject'] = True  # 根据附加说明主体默认为True

    # 2. 标记触发条件（直接映射布尔列）
    df['cu_19_5_condition'] = df['相关股东前三个月内存在减持情形']

    # 3. 标记约束条件（高送转方案判断）
    # 高送转定义为实施股票股利或公积金转增股本
    has_high_send = df['是否实施股票股利'] | df['是否公积金转增股本']
    df['cu_19_5_constraint'] = ~has_high_send

    return df

In [ ]:
df = check_cu_19_5(df)
df

---

## cu_19_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 相关股东在后三个月存在减持计划 |
| constraint | 不得披露高送转方案 |
| contextual_info | 相关股东指公司的提议股东和控股股东及其一致行动人、董事、监事及高级管理人员 |
| note | 主体默认为True, 不区分在上市前还是上市后 |
| relation | refer_to |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2443 |
| completion_tokens | 3460 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_6(df):
    '''
    检查cu_19_6的合规性：
    主体：上市公司（所有行标记为True）
    条件：相关股东后三个月存在减持计划
    约束：未披露高送转方案（未实施股票股利或转增股本）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_19_6_subject'] = True  # 所有上市公司主体均有效
    
    # 标记触发条件
    df['cu_19_6_condition'] = df['相关股东后三个月存在减持计划']
    
    # 标记约束条件（高送转方案判断）
    # 高送转定义为实施股票股利或资本公积转增股本
    has_high_send = df['是否实施股票股利'] | df['是否公积金转增股本']
    df['cu_19_6_constraint'] = ~has_high_send
    
    return df

In [ ]:
df = check_cu_19_6(df)
df

---